<a href="https://colab.research.google.com/github/AyanBhunia/Super-brain-ML/blob/master/Music_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing important libraries


*   glob lets you match specified patterns.
*   music21 is a toolkit that let you work with music files. Understand them, process them and write them.



In [4]:
import glob
from music21 import converter, instrument, note, chord


1. **converter**:  It is used to load music from various file formats music21.stream slass
2. **instrument**:  It is used to gather the information about the instruments used in the music file.
3. **note**: It is a class in music21 toolkit to gather information about notes and to create Notes in a music file.
4. **chord**: It is a class in music21 toolkit to gather information about Chords and to create Chords in a music file.



# Creating the dataset


Here we will iterate over each **midi file** in my dataset and extract the information about the notes and chords of the music file. We will read the file as a **sequence of Notes and Chords**. These sequences are going to be my dataset.

In [5]:
!unzip "/content/drive/My Drive/super-brain/midi_songs.zip" -d "/content/"

Archive:  /content/drive/My Drive/super-brain/midi_songs.zip
replace /content/midi_songs/0fithos.mid? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/midi_songs/0fithos.mid  
  inflating: /content/midi_songs/8.mid  
  inflating: /content/midi_songs/ahead_on_our_way_piano.mid  
  inflating: /content/midi_songs/AT.mid  
  inflating: /content/midi_songs/balamb.mid  
  inflating: /content/midi_songs/bcm.mid  
  inflating: /content/midi_songs/BlueStone_LastDungeon.mid  
  inflating: /content/midi_songs/braska.mid  
  inflating: /content/midi_songs/caitsith.mid  
  inflating: /content/midi_songs/Cids.mid  
  inflating: /content/midi_songs/cosmo.mid  
  inflating: /content/midi_songs/costadsol.mid  
  inflating: /content/midi_songs/dayafter.mid  
  inflating: /content/midi_songs/decisive.mid  
  inflating: /content/midi_songs/dontbeafraid.mid  
  inflating: /content/midi_songs/DOS.mid  
  inflating: /content/midi_songs/electric_de_chocobo.mid  
  inflating: /content/midi_songs/E

In [6]:
notes=[]
for file in glob.glob("/content/midi_songs/*.mid"):
  midi = converter.parse(file) #this will convert the file intp music.21 stream.score objects.
  parts = instrument.partitionByInstrument(midi)# this will create partition into a Part for each unique Instrument.
  notes_to_parse = parts.parts[0]
  #this will have information about the notes,chords and rests in the song.
  #now we need to extract the information of notes and chords.
  for element in notes_to_parse:
    if isinstance(element, note.Note):
      #print(str(element.pitch))
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      #print(str(element.normalOrder))
      notes.append('.'.join(str(n) for n in element.normalOrder))

Now we have the notes and chords information of each song in the list named **'notes'**. Let's have a look at the list.

In [7]:
notes

['D2',
 'D2',
 'D2',
 'D2',
 'D2',
 'D2',
 'C2',
 'C2',
 'D2',
 'D2',
 'D2',
 'D2',
 'D2',
 'D2',
 'C2',
 'C2',
 '2',
 'E4',
 'D2',
 'F4',
 'D2',
 'A4',
 'D2',
 '0.3',
 'B-4',
 'E-2',
 'G4',
 'E-2',
 'B-4',
 'E-2',
 'A4',
 'D2',
 'D2',
 'D4',
 'D2',
 'D2',
 'E-2',
 '3.7',
 '3.5',
 '3',
 'D4',
 'D2',
 'D2',
 'A3',
 'D2',
 'D2',
 'E-2',
 '3.9',
 '10.3',
 '0.3',
 '9.2',
 '7.0',
 'D2',
 '5.10',
 '2.4.9',
 '2.7',
 '0.3.5',
 '10.3',
 '9.2.3',
 '0.3.7',
 '2',
 'E4',
 'D2',
 'F4',
 'D2',
 'A4',
 'D2',
 '0.3',
 'B-4',
 'E-2',
 'G4',
 'E-2',
 'B-4',
 'E-2',
 'A4',
 'D2',
 'D2',
 'D4',
 'D2',
 'D2',
 'G4',
 'E-2',
 'E-2',
 '3.5',
 '3',
 'D4',
 'D2',
 'D2',
 'D2',
 'D2',
 'E-2',
 '3.9',
 '10.3',
 '0.3',
 '9.2',
 'D2',
 '9.2',
 '9.2',
 'D2',
 '9.2',
 '0.5',
 'D2',
 '0.5',
 '0.5',
 'D2',
 '0.5',
 '5.10',
 'E-2',
 '5.10',
 '5.10',
 'E-2',
 '5.10',
 '10.3',
 'E-2',
 '10.3',
 '10.3',
 'E-2',
 '10.3',
 '9.2',
 'D2',
 '9.2',
 'D2',
 'D3',
 '10.3',
 '0.5',
 'E-2',
 '0.5',
 'E-2',
 'E-3',
 '10.3',
 '9.2',


In [8]:
len(notes)

57177

This means we have 57177 notes and chords in all the songs combinely. Now we need to Form our dataset with this

The idea to generate the dataset is, one input will have 100 notes+chords and the 101th note or chord is our output for that input. Then the second input output will be generated by shifting the same task for next 100 notes and chords.


**For Example:**
 * from 0th index till 99th index the sequence of notes and chords is the input, the note or chord present at the 100th index is the output for this particular input.

 * from 100th index till 199th index the sequence of notes and chords is the input, the note or chord present at the 200th index is the output for this particular input.

 And so on.

Additionally, we will also have to **encode them**. As the model doesn't understand anything present in **object or str** field.

In [9]:
import numpy 
from keras.utils import np_utils

n_vocab = len(set(notes))#to get the number of unique data present in notes
sequence_length = 100  #You can keep it higher or lower
# get all pitch names
pitchnames = sorted(set(item for item in notes)) #getting various unique data present in notes
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])


# Data Preprocessing

Now we need to reshape and normalize the input data. Also we will one hot encode the output data.

In [10]:
n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
#normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

# Buiding the Model

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation


In [12]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

# Training the Model

In [18]:
model.fit(network_input, network_output, epochs=120, batch_size=20)

Epoch 1/120
2854/2854 [==============================] - 89s 31ms/step - loss: 4.6577
Epoch 2/120
1335/2854 [=============>................] - ETA: 47s - loss: 4.7445

KeyboardInterrupt: ignored

# Saving the Model

In [ ]:
  model.save_weights('music125.h5')


In [ ]:
with open('model_architecture125.json','w') as f:
  f.write(model.to_json())

# Restoring the Model

In [ ]:
from keras.models import load_model
from keras.models import model_from_json

In [ ]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# Model reconstruction from JSON file
with open('/content/model_architecture125.json', 'r') as f:
    model = model_from_json(f.read())

In [ ]:

# Load weights into the new model
model.load_weights('/content/music125.h5')


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

# Testing the model

In [ ]:
type(network_input[0])

numpy.ndarray

In [ ]:
start = numpy.random.randint(0, len(network_input)-1)
b=network_input[start]

In [ ]:
b

array([[0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.89106145],
       [0.99162011],
       [0.59217877],
       [0.88826816],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.99162011],
       [0.88826816],
       [0.59217877],
       [0.98882682],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.98882682],
       [0.88826816],
       [0.59217877],
       [0.88547486],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.32402235],
       [0.59217877],
       [0.98882682],
       [0.59217877],
       [0.592

In [ ]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = (network_input[start])
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = (prediction_input) / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern=numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]


In [ ]:
prediction_output

['2.5',
 '0.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '2.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',
 '0.5',
 '0.5',
 '0.5',
 '2.5',
 '0.5',
 '2.5',
 '0.5',
 '0.5',
 '2.5',
 '2.5',


# Converting output into midi file

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
import music21
midi_stream = music21.stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'